# Roster Alignment

The purpose of this notebook is to unify:

1. Sakai's Gradebook exports ONYEN and PID
2. Connect Carolina Maps PID->Email
3. Gradescope's Email-based mapping of students

This is an unfortunate case of three systems choosing to ID based on three different keys.

The outcome of this notebook should be a single CSV that contains, for every student in the course, a row with:

* PID
* ONYEN
* Email
* Section

A separate notebook is used for pulling grades. This is simply trying to produce a single dataset with IDs for each of the systems involved.

In [6]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import warnings
import re
# Ignore noise about Excel spreadsheet styling on the next command
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

# Connect Carolina Rosters - Using Connect Carolina's Roster Tool (from the sidebar, not the home page)
# Go to Student Admin > Class Roster > Select Class
# Top-left corner select: Related Content > Class Roster Email > CSV File
cc_rosters = {
    "001": "./22f-001-roster-cc.csv",
    "002": "./22f-002-roster-cc.csv"
}

# Sakai Rosters - Using Sakai's Roster Tool, Export Excel Sheets for Each Section
# Dictionary Maps Section -> Excel File Path
# If Sakai rosters are already unified, bring them here
sakai_rosters = {
    "001,002": "./gradebook_export.csv",
}

# Gradescope Rosters - Export from Roster Tab
# If Gradescope rosters also unified, single entry here.
gs_rosters = {
    "001,002": "./COMP110_Fall_2022_roster.csv",
}

## Sakai Gradebook Ingestion

Enable the gradebook on Sakai in order to get an export that contains both the PID and the ONYEN of enrolled students (the roster export only contains ONYEN).

In [7]:
sakai_roster = None

for section in sakai_rosters:
    csv_file = sakai_rosters[section]
    current_roster = pd.read_csv(csv_file)
    # current_roster["Section"] = section
    names = current_roster["Name"].str.extract(r'(?P<Last>[^,]+) ?[^,]*, (?P<First>[^,]+)')
    current_roster = pd.concat([current_roster, names], axis=1)
    current_roster = current_roster.rename(columns={"Student ID": "ONYEN"})
    current_roster = current_roster[["Name", "Last", "First", "PID", "ONYEN"]]
    sakai_roster = current_roster if sakai_roster is None else pd.concat([sakai_roster, current_roster])

sakai_roster = sakai_roster.reset_index().drop(["index"], axis=1)
sakai_roster

,Name,Last,First,PID,ONYEN
0,"Abdul-Haqq, Ayah",Abdul-Haqq,Ayah,730615311,abd
1,"Ackron, Andreas",Ackron,Andreas,730561624,ackron
2,"Adiga, Prithvi",Adiga,Prithvi,730480357,padiga
3,"Adrias, Carter",Adrias,Carter,730555728,cadrias
4,"Agada, Uredoojo",Agada,Uredoojo,730473059,uredo
...,...,...,...,...,...
689,"Zhu, Sonia",Zhu,Sonia,730472938,sosoni
690,"Zhu, Yan",Zhu,Yan,730615250,zhuyan
691,"Zolotor, Molly",Zolotor,Molly,730552348,mzolotor
692,"Zou, Raymond",Zou,Raymond,730581613,raymondz


## Connect Carolina Ingestion

Here we're reading in Connect Carolina's rosters.

In [9]:
cc_roster = None

for section in cc_rosters:
    csv_file = cc_rosters[section]
    current_roster = pd.read_csv(csv_file)
    current_roster = current_roster[current_roster["Status Note"] != "Withdrawn/Drop"]
    # Extra first name last name (and ignore occassional middle name...)
    names = current_roster["Name"].str.extract(r'(?P<Last>[^, ]+) ?[^,]*,(?P<First>[^, ]+)')
    names["First"] = names["First"].str.lower().str.replace(r'[^a-z]', '', regex=True)
    names["Last"] = names["Last"].str.lower().str.replace(r'[^a-z]', '', regex=True)
    current_roster = pd.concat([current_roster, names], axis=1)
    current_roster = current_roster.rename(columns={"ID": "PID"})
    current_roster = current_roster.drop(["Status Note", "FERPA Full Indicator"], axis=1)
    current_roster["Section"] = section
    current_roster = current_roster.reindex(columns=["Name", "Last", "First", "PID", "Email", "Section"])
    current_roster["Email"] = current_roster["Email"].str.lower()
    cc_roster = current_roster if cc_roster is None else pd.concat([cc_roster, current_roster])   
    
cc_roster = cc_roster.reset_index().drop(["index"], axis=1)
cc_roster

,Name,Last,First,PID,Email,Section
0,"Agada,Uredoojo Aluana",agada,uredoojo,730473059,uagada@unc.edu,001
1,"Agbebi,Morounfoluwa David",agbebi,morounfoluwa,730480375,magbebi@unc.edu,001
2,"Agyemang,Dylan",agyemang,dylan,730547610,agdylan@unc.edu,001
3,"Ahmed,Sara Aliya",ahmed,sara,730461917,sahmed@unc.edu,001
4,"Akshinthala (Uk-Shin-Thuh-La),Phanisree (Puh-N...",akshinthala,phanisree,730516244,pakshinthala@unc.edu,001
...,...,...,...,...,...,...
689,"Yi,Jason H",yi,jason,730604615,jasonyi@unc.edu,002
690,"Zareiesfandabadi,Borna",zareiesfandabadi,borna,730622413,bornaz@unc.edu,002
691,"Zarrouk,Marwan",zarrouk,marwan,730581174,marwanz@unc.edu,002
692,"Zeresenai,Somit",zeresenai,somit,730472896,szeresenai@unc.edu,002


## Gradescope Ingestion

In [11]:
gs_roster = None

for section in gs_rosters:
    csv_file = gs_rosters[section]
    current_roster = pd.read_csv(csv_file)
    # current_roster["Section"] = section
    # Bring back in once fully sync'ed and rosters settle:
    # current_roster = current_roster[~current_roster["section_name"].isna()]
    current_roster = current_roster.drop(["SID", "Role"], axis=1)
    current_roster = current_roster.rename(columns={"Last Name": "Last", "First Name": "First"})
    current_roster = current_roster.reindex(columns=["Last", "First", "Email"])
    current_roster["Email"] = current_roster["Email"].str.lower()
    gs_roster = current_roster if gs_roster is None else pd.concat([gs_roster, current_roster])

gs_roster = gs_roster.reset_index().drop(["index"], axis=1)
gs_roster

,Last,First,Email
0,Jain,Aashvi,aashvij@unc.edu
1,Kondru,Abhay,akondru@unc.edu
2,Mayreddy,Abhinav,avvmay@unc.edu
3,Seo,Abigail,eunbin@unc.edu
4,Lujan,Abisai,lujana@live.unc.edu
...,...,...,...
703,Sanders,Zahria,zzelisia@email.unc.edu
704,Kamdar,Zaid,zkamdar@unc.edu
705,Zeng,Zhuohao,zhuohao_zeng@unc.edu
706,Wang,Ziqiao,wzq@unc.edu


## Unifying the Data

Let's use Connect Carolina as the authoritative roster source. From there we will try to find the matching Sakai and Gradescope entries.

In [13]:
sakai_by_pid = sakai_roster.set_index("PID")
cc_by_pid = cc_roster.set_index("PID")
gs_by_email = gs_roster.set_index("Email")
gs_by_name = gs_roster.set_index(["Last", "First"])

cc_by_pid

,Name,Last,First,Email,Section
PID,,,,,
730473059,"Agada,Uredoojo Aluana",agada,uredoojo,uagada@unc.edu,001
730480375,"Agbebi,Morounfoluwa David",agbebi,morounfoluwa,magbebi@unc.edu,001
730547610,"Agyemang,Dylan",agyemang,dylan,agdylan@unc.edu,001
730461917,"Ahmed,Sara Aliya",ahmed,sara,sahmed@unc.edu,001
730516244,"Akshinthala (Uk-Shin-Thuh-La),Phanisree (Puh-N...",akshinthala,phanisree,pakshinthala@unc.edu,001
...,...,...,...,...,...
730604615,"Yi,Jason H",yi,jason,jasonyi@unc.edu,002
730622413,"Zareiesfandabadi,Borna",zareiesfandabadi,borna,bornaz@unc.edu,002
730581174,"Zarrouk,Marwan",zarrouk,marwan,marwanz@unc.edu,002


In [14]:
for pid in sakai_by_pid.index:
    sakai = []

    sakai_student = sakai_by_pid.loc[pid]
    cc_student = cc_by_pid.loc[pid]
    sakai_by_pid.loc[pid, ["CC Email"]] = cc_student["Email"]
    sakai_by_pid.loc[pid, ["Section"]] = cc_student["Section"]

    try:
        gs_student = gs_by_email.loc[cc_student["Email"]]
        sakai_by_pid.loc[pid, ["GS Email"]] = gs_student.name
    except:
        gs_student = gs_by_name.loc[sakai_student["Last"]].loc[sakai_student["First"]]
        sakai_by_pid.loc[pid, ["GS Email"]] = gs_student["Email"]
    
sakai_by_pid.to_csv("roster_merged.csv")
sakai_by_pid

,Name,Last,First,ONYEN,CC Email,Section,GS Email
PID,,,,,,,
730615311,"Abdul-Haqq, Ayah",Abdul-Haqq,Ayah,abd,abd@unc.edu,002,abd@unc.edu
730561624,"Ackron, Andreas",Ackron,Andreas,ackron,ackron@unc.edu,002,ackron@unc.edu
730480357,"Adiga, Prithvi",Adiga,Prithvi,padiga,padiga@unc.edu,002,padiga@unc.edu
730555728,"Adrias, Carter",Adrias,Carter,cadrias,cadrias@unc.edu,002,cadrias@unc.edu
730473059,"Agada, Uredoojo",Agada,Uredoojo,uredo,uagada@unc.edu,001,uagada@unc.edu
...,...,...,...,...,...,...,...
730472938,"Zhu, Sonia",Zhu,Sonia,sosoni,sosoni@ad.unc.edu,001,sosoni@ad.unc.edu
730615250,"Zhu, Yan",Zhu,Yan,zhuyan,zhuyan@unc.edu,001,zhuyan@unc.edu
730552348,"Zolotor, Molly",Zolotor,Molly,mzolotor,mzolotor@unc.edu,001,mzolotor@unc.edu
